<a href="https://colab.research.google.com/github/davidnoone/PHYS332_FluidExamples/blob/main/06_BlasiusBoundaryLayer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Blasuis boundary layer

Cosider steady flow above a plate. Some distance away from the plate the flow has horizontal velocity U. The plate is infinitely long in the y direction, leading to a 2 dimensional problem in the x-z plane. The flow is laminar, but viscous. The equations describing the flow above a plate are:

$$
  \frac{\partial u}{\partial x} +   \frac{\partial w}{\partial z} = 0
$$

and 
$$
 u\frac{\partial u}{\partial x} + w \frac{\partial u}{\partial z} =
   -\frac{1}{\rho}\frac{\partial p}{\partial x} + \nu \frac{\partial^2 u}{\partial z^2} 
$$

For now, let's consider pressure to be uniform (pressure gradient is zero). For steady incompressible flow, a streamfunction is defined such that

$$ 
u = \frac{\partial \psi}{\partial z}
$$

$$ 
w = -\frac{\partial \psi}{\partial z}
$$


This immediatly satisfies the continuity equation and the momentum equation can be rewitten

$$
\frac{\partial \psi}{\partial z}
  \frac{\partial^2 \psi}{\partial x \partial z} - 
\frac{\partial \psi}{\partial x}
  \frac{\partial^2 \psi}{\partial z^2} =
\nu    \frac{\partial^3 \psi}{\partial z^3}
$$

This system can be solved given appropriate boundary conditions.


# Non-dimensional solution 

The form of the eauations suggest a non-dimensional vertical distance:

$$
\eta = \sqrt{ \frac{U}{\nu x}} z
$$

The streamfunction is written in a non-dimensional form as:

$$
  \psi(x,z) = \sqrt{2U \nu x} f(\eta)
$$

where f = f(\eta) is the non-dimensional streamfunction. With this definition, the Blasius problem can be reduced to a non-dimensional equation for streamfunction:

$$ 
f'''(\eta) + f(\eta)f''(\eta) = 0
$$

This third order equation needs three boundary conditions:
$$
f(0) = 0
$$

$$
f'(0) = 0
$$
$$
f'(\inf) = 1
$$

The quantity $\eta$ is refered to as a similarity parameter. Notice that it involves both x and z, and suggests the solution at all points x are "similar" for a particular depth z. Therefore it allos us to transform the 2d problem $\psi(x,z)$ into a 1d problem $f(\eta)$.

### Solution method
Solving nonlinear ODEs in general requires numerical methods. Here it is convinenient to make a "geuss" and check if the solutions meet the boundary conditions, and if not, guess again. The sequence of guessing can be done via bisection for a solution starting with a guess which is clearly high and a guess which is clearly low. Several iterations allow the algorithm to hone in on the solution to within a specific tolerance (could be numerical precision.



## Recovering results

The dimensional quantities can be recovered noting that

$$
u(x,z) = \frac{\partial \psi}{\partial z} = U f'
$$


$$
w(x,z) = -\frac{\partial \psi}{\partial z} = \frac{1}{2}U (\eta f' - f)/\sqrt{Re}
$$

Where $Re$ is the Reynolds number 

$$ 
Re = \frac{UL}{\nu}
$$

and we set L the (vertical) length scale.

## Boundary layer depth

We define the depth of the boundary layer to be the height corresponding to where $u = 0.99 U$. This is a practical definition. This is the case when $\eta = 4.9$. 

$$
\delta(x) = \frac{4.9 x}{\sqrt{Re}}
$$

## Surface stress
The surface stress, a meansure of total frictonal drag, is expressed in the normal way as proportional to the shear:

$$ 
\tau_s = \mu \left(\frac{\partial u}{\partial z}\right)_{z=0}
$$

So from the definitions above:
$$ 
\tau_s = \mu Uf'' \sqrt{\frac{U}{\nu z}}
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#  Intégration de l'équation de Blasius (Vincent Legat, Avril 2020)

# Define the Blasius equation f''' + f f'' = 0
#
#           dudt(t) = v(t)
#           dvdt(t) = w(t)
#           dwdt(t) = -u(t)*w(t) 
#
def f(u):
  dudt =   u[1]
  dvdt =   u[2] 
  dwdt = - u[0] * u[2]  
  return np.array([dudt,dvdt,dwdt])
 
# 4th order Runge-Kutta step
def rungekutta(a,h):
  imax = int(5/h)
  U = np.zeros((imax+1,3))
  U[0,:] = np.array([0,0,a])
  
  for i in range(imax):  
    K1 = f(U[i,:]       )
    K2 = f(U[i,:]+K1*h/2)
    K3 = f(U[i,:]+K2*h/2)
    K4 = f(U[i,:]+K3*h  )
    U[i+1,:] = U[i,:] + h*(K1+2*K2+2*K3+K4)/6     
  return U
 
# Solve ODE by bisection method ("shooting method")
def shoot(a,h):
  U = rungekutta(a,h)
  return (1.0-U[-1,1])
  
def blasius(h,tol):
  nmax = 40
  a = 0
  b = 1
  fa = shoot(a,h)
  fb = shoot(b,h)
  delta = (b-a)/2
  n = 0
  if (fa*fb > 0) :
    raise RuntimeError('Bad initial interval') 
  while (abs(delta) >= tol and n <= nmax):
    n = n + 1
    delta = (b-a)/2
    x = a + delta
    fx = shoot(x,h)
    print(" x = %14.7e (Estimated error %13.7e at iteration %d)" % (x,abs(delta),n))
    if (fx*fa > 0) :
      a = x
      fa = fx
    else :
      b = x
      fb = fx
  if (n > nmax) :
    raise RuntimeError('Too many iterations') 
  print('Blasius:',x)
  return x
 


In [ ]:
tol = 1e-7
h   = 0.1

Xmax = 5
imax = int(Xmax/h)

# Set up the 1d coordinate, which is "eta"
X = np.arange(imax+1)*h

# Solve the Differential equation
a = blasius(h,tol)

# integrate vertically to construct the f, f' and f'' profile
U = rungekutta(a,h)

print(" ============ Final value for f''(0) = %.4f " % U[0,2])
 
fig = plt.figure()
plt.plot(U[:,1]*X - U[:,0],X,'-b')
plt.plot(U[:,1]           ,X,'-r')
 
plt.text(0.80, 1.5,r'$\frac{u}{U}$',color='red',fontsize=18)
plt.text(0.20, 1.5,r"$\frac{w}{U \delta'}$",color='blue',fontsize=18)
plt.text(0.05, 4.9,r'$\eta$',color='black',fontsize=15)

plt.title('Blasius solution')
plt.xlabel('$f(\eta)$')
plt.ylabel('$\eta$')

#plt.savefig('blasius.png')
print("Finished")

In [ ]:
# Plotting


